# Make OlthoPhoto and PhaseAngleImage

In [21]:
import csv
import cv2
import math
import tifffile
import pandas as pd
import numpy as np
from numpy import loadtxt
import xml.etree.ElementTree as ET # XMLファイル用
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
import colorsys

# 処理時間計測
import time
from tqdm import tqdm

# 並列処理
import os
import concurrent.futures as confu

# 太陽
import ephem   
import pyproj
from pyproj import Transformer

# 画像から時刻取得
import piexif
import re
#from PIL import Image
#from PIL.ExifTags import TAGS
#import PIL.Image

# 画像を表示するためのライブラリ ######################
from IPython.display import Image, display_png
import matplotlib.pyplot as plt
from PIL import Image

# Jupyterでインライン表示するための宣言
%matplotlib inline

## 初期設定パラメータ ４ グループ

In [22]:
################################################################
###############   1/4 手入力が必要な初期設定　 #################
################################################################
# 対象観測シーンIDとデータディレクトリは？
SceneID = "Kotogahama-20231228-1153-P1"  #Toitajima-20231013-1126-P2
                                         #Kotogahama-20231218-1109-P1
                                         #Kotogahama-20231228-1153-P1
                                         #Kotogahama-20240327-1302-P1
CameraName = "P1"

# 原画像の近傍平均処理範囲は？
a         = 15      # a*2+1 が平均される縦横の範囲
PixelArea = a*2+1   ## 戸板島、GSD=0.50cm
                    ## 琴ヶ浜 ##
                    #1. 11:09 高度100mで、堤防から汀線までを撮影 GSD=1.26cm  OL 80% , SL 80&
                    #2. 11:36 高度12mで、ROIを撮影 GSD=0.12cm  OL 90% , SL80%
                    #3. 11:53 高度40mで、ROIを撮影 GSD=0.50cm  OL 90% , SL90%
                        #0.05m:a=4, 0.1m:a=9, 0.2m:a=19, 0.5m:a=49, 1.0m:a=99
                    #4. 12:04 高度12mで、ROIを撮影 GSD=0.12cm  OL 90% , SL80%
                        # 0.05m:a=19, 0.1m =40, 0.2m:a=82, 0.5m:a=2

                    # Kotogahama 5=a2, 7=a3,9=a4,11=a5,13=a6,15=a7,17=a8,19=a9,21=a10,23=a11,25=a12,27=a13,29=a14
                    # Kotogahama 31=a16, 

# イメージ拡張子は？
ImageExtension = "tif"

# ====================== 対象エリアの設定 ==========================
# 戸板島
#xmin,xmax,ymin,ymax,zmin,zmax = 18186, 18274, 65805, 65895, 55,64
# 琴ヶ浜 #1153-P1
xmin,xmax,ymin,ymax,zmin,zmax = 26251,26320 , 57320,57379 , 30,40 #Kotogahama-20231228-1153-P1
# 琴ヶ浜 #1204-P2
#xmin,xmax,ymin,ymax,zmin,zmax = 26265,26310 , 57325,57372 , 34,37 #Kotogahama-20231228-1204-P2
# 琴ヶ浜 20240327-1302-P1
#xmin,xmax,ymin,ymax,zmin,zmax = 27345,27490 , 57297,57397 , 25,36 #Kotogahama-20240327-1302

# 点群の間隔(単位:m)
Interval   =  0.05 #0.05 #0.1 #0.2 #0.5 #1.0  
IntervalID = "005-0"+ str(PixelArea) #005  #010 #020 #050 #100

# 投影面の2次元配列生成
xsize = int(round((xmax-xmin)/Interval))
ysize = int(round((ymax-ymin)/Interval))

print("原画像DNの取得範囲：",(PixelArea),"×",(PixelArea))
print("オルソ画像の分解能：",Interval,"m")


################################################################
################  2/4 使用するデータのパス設定  ################
################################################################
# 入出力デバイス選択
Win  = "/mnt/i/"
Mac  = "/Volumes/RM1/"
Lin  = "/media/kut/RM1/"
OSd  = Lin

# ディレクトリのパス設定
Dir1 = OSd  + "003_RESEARCH-河川計測_4_BRF/" +SceneID+ "/"
Dir2 = Dir1 + "O_IntermediateData_" + str(IntervalID)+ "/"
Dir3 = Dir1 + "O_Tif/"
Dir4 = Dir1 + "O_OlthoPhoto/"
Dir5 = Dir1 + "O_ThetaImage/"
Dir6 = Dir1 + "O_ObservationAngleImage/"
Dir9 = Dir1 + "I_Raw/"

# 入力ファイルのパス設定
IFile1 = Dir1 +SceneID+ "_CameraOmegaPhiKappa.txt"  # 入力１：カメラデータ
IFile2 = Dir1 +SceneID+ "_CameraXML.xml"
IFile3 = Dir2 +SceneID+ "_DSMtliming.csv"           # 入力２：投影面になるDSMデータ（DSM）
IFile4 = Dir1 +SceneID+ "_ListImages.csv"           # 入力３：原画像のリスト
IFile5 = Dir1 +SceneID+ "_ListImages-INSIDE.csv"    # 入力４：原画像のリスト（範囲内）
#File6 = Dir2 +SceneID+ "_ShadowTliming.tif"        # 入力５：日向日陰シミュレーション

# IFile5を作成（IFile4をオリジナルとして残すため）
df = pd.read_csv(IFile4, header=None, encoding='utf8')  # CSVファイルを読み込み
df.to_csv(IFile5, index=False, header=False)           # CSVファイルに保存


##################################################################
##############  3/4 カメラデータ・原画像・投影面の読込　##########
##################################################################

# IF:カメラデータの読み込み
camera_data_pd = pd.read_csv(IFile1, skiprows=2, sep='\t', names=("#PhotoID","X","Y","Z","Omega","Phi","Kappa","r11","r12","r13","r21","r22","r23","r31","r32","r33"))

# IF:投影面となるDSM（点群）読み込み(Pandasを利用。Numpyの場合は1行で書けるけど、今は取り回しを鑑みてPandasにしている)
P_pd1 = pd.read_csv(IFile3, skiprows=1, names=("x","y","z","r","g","b","n","i","j","k"))
P_pd2 = P_pd1.drop(["r","g","b","n"],axis=1)
P_np  = P_pd2.values

# 原画像全部処理
PhotoIDs = []
with open(IFile4, "r", encoding="utf-8-sig") as f: # 画像データを読み込み
    Photolist = f.readlines()
for i in Photolist:
    PhotoIDs.append(i.rstrip())


#################################################################
###############  4/4 プログラムテスト用の設定項目　##############
#################################################################

# １枚だけ試したい時は原画像データを指定
# PhotoIDs = ["DJI_20231013112640_0045"]
# PhotoIDs = ["DJI_0283"]

print("画像枚数:",len(PhotoIDs),",  ","DSM縦:",xsize,",  ","DSM",ysize,)

原画像DNの取得範囲： 31 × 31
オルソ画像の分解能： 0.05 m
画像枚数: 148 ,   DSM縦: 1380 ,   DSM 1180


## Class & Function

### センサClassの作成

In [23]:
# カメライメージセンサの設定
class SensorParameter: # 手入力で変数を定義する
    def __init__(self,Sensor):
        # 撮影画像
        self.SensorID = Sensor

        # カメラ位置姿勢情報の取得
        XML = ET.parse(IFile2)     # XMLファイルの読み込み
        xml = XML.getroot()
        
        pixel_size        = [str(part.get('value')) for part in xml.iter('property')]
        self.pixel_size   = float(pixel_size[0]) / 1000                  # １画素の物理的な大きさ(単位:m)
        
        cx                = [str(part.text) for part in xml.iter('cx')]
        cy                = [str(part.text) for part in xml.iter('cy')]
        self.cX, self.cY  = float(cx[0]), float(cy[0])                   # ピクセル座標系 における主点
        
        i_rX              = [str(part.get('width')) for part in xml.iter('resolution')]
        i_rY              = [str(part.get('height')) for part in xml.iter('resolution')]
        self.i_rX         = int(i_rX[0])
        self.i_rY         = int(i_rY[0])
        
        self.sensor_sizeX = self.pixel_size * self.i_rX
        self.sensor_sizeY = self.pixel_size * self.i_rY
        
        self.focalLength  = [str(part.text) for part in xml.iter('f')] 
        self.focal_length = float(self.focalLength[0]) * self.pixel_size
        self.iFov         = math.sqrt((self.sensor_sizeX*self.sensor_sizeY)/(self.i_rX*self.i_rY))/self.focal_length*1000
        
        # オクルージョンで使用するZの配列作成
        self.VoxelK = np.zeros((ysize, xsize)).astype(int)
        for p in P_np:
            i, j, k = int(p[3]),int(p[4]),int(p[5])
            self.VoxelK[j,i] = k

        self.xstart, self.ystart = 0, 0
        self.xstop,  self.ystop  = xsize-1, ysize-1

    def printInfo(self):
        print("\n","[ Sensor Info","]")
        print("Sensor Nmae:",self.SensorID, "  Pixel Width:",self.i_rX, "  Pixel Hight:",self.i_rY)
        print("Pixel Size:",'{:.13f}'.format(self.pixel_size), end=" ")
        print("Cx:",'{:.5f}'.format(self.cX)," Cy:",'{:.5f}'.format(self.cY))
        print("Sensor SizeX(m)",'{:.5f}'.format(self.sensor_sizeX), "  Sensor SizeY(m)",'{:.5f}'.format(self.sensor_sizeY))
        print("Focal Length:",'{:.5f}'.format(float(self.focalLength[0])), " Camera iFOV:",'{:.3f}'.format(self.iFov))

### 原画像Classの作成

In [24]:
# 原画像とカメラ位置姿勢情報
class CameraParameterEach: # 手入力で変数を定義する
    def __init__(self, photoID):

         # 撮影画像
        self.photoID = photoID
        
        # 入力ファイル
        self.IFile1 = Dir3 + photoID + "." + ImageExtension                   # 入力2 - 原画像
        self.IFile2 = Dir9 + photoID + ".DNG"

        # 出力ファイル
        self.OFile1 = Dir4 + "VI-"       +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # 出力1 - オルソ画像
        self.OFile2 = Dir5 + "PA-"       +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # 出力2 - 位相角画像
        self.OFile3 = Dir6 + "OA-HSV-"   +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # 出力3 - 観測角HSV
        self.OFile4 = Dir6 + "OA-RGB-"   +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # 出力4 - 観測角RGB
        self.OFile5 = Dir6 + "OA-Normal" +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # 出力5 - 観測角Normal


        # 原画像の読み込
        #img      = cv2.imread(self.IFile1) # 読み込み
        #self.img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)       # RGBの順番入れ替え
        self.img  = tifffile.imread(self.IFile1)
        self.img_R,self.img_G,self.img_B = np.array(self.img[:,:,0]), np.array(self.img[:,:,1]), np.array(self.img[:,:,2])
        self.h, self.w, band             = self.img.shape

        # カメラデータの取得準備（OPK）
        self.num = camera_data_pd.loc[camera_data_pd['#PhotoID'] == photoID].index[0] # カメラの行番号を取得

        # カメラの姿勢情報
        self.x  = camera_data_pd["X"][self.num]
        self.y  = camera_data_pd["Y"][self.num]
        self.z  = camera_data_pd["Z"][self.num]
        self.Ci = round(((self.x-xmin)/Interval),0).astype(int)
        self.Cj = round(((ymax-self.y)/Interval),0).astype(int)
        self.Ck = round(((self.z-zmin)/Interval),0).astype(int)
        O       = camera_data_pd["Omega"][self.num]
        P       = camera_data_pd["Phi"][self.num]
        K       = camera_data_pd["Kappa"][self.num]
        
        # 度数から弧度
        rad_O = math.radians(float(O))
        rad_P = math.radians(float(P))
        rad_K = math.radians(float(K))

        # 三角関数の計算
        sin_O = math.sin(-float(rad_O))
        cos_O = math.cos(-float(rad_O))
        sin_P = math.sin(-float(rad_P))
        cos_P = math.cos(-float(rad_P))
        sin_K = math.sin(-float(rad_K))
        cos_K = math.cos(-float(rad_K))
        
        # 共線条件式（途中まで）
        arr2      = np.array([[cos_K,-sin_K,0], [sin_K,cos_K,0], [0,0,1]])  # kappa
        arr3      = np.array([[cos_P,0,sin_P], [0,1,0], [-sin_P,0,cos_P]])  # phi
        arr4      = np.array([[1,0,0], [0,cos_O,-sin_O], [0,sin_O,cos_O]])  # omega
        arr5      = np.dot(arr2, arr3)
        self.arr6 = np.dot(arr5, arr4)
                
        # 撮影している範囲の計算（対地高度情報を入力）
        self.grand_reso   = siFov * (self.z - 135.00)   # 対象面の高さを135mと仮定した
        self.areaW, self.areaH = self.grand_reso * si_rX / 1000, self.grand_reso * si_rY / 1000
        self.iXmin, self.iXmax = self.x - (self.areaW/2), self.x + (self.areaW/2)
        self.iYmin, self.iYmax = self.y - (self.areaH/2), self.y + (self.areaH/2)

        ###################################
        #############  位相角  #############
        ###################################
        # 座標系の設定(新しいプログラムに変更,2024/01)
        #p1 = pyproj.Proj("EPSG:2446")  # 変換元の座標：日本測地系4系
        #p2 = pyproj.Proj("EPSG:4326")  # 変換後の座標系：緯度経度
        transformer  = Transformer.from_crs("EPSG:2446", "EPSG:4326")

        # 撮影時刻の取得
        Date = self.PhotoTakenTime(self.IFile2)
        self.Date = Date # 無くてもいい
        
        # [[注意]]：カメラ位置における太陽をシミュレーションする（self.xとyはcxとcy / 投影面の方が精度がいいかも）
        Lat, lon             = transformer.transform(self.x, self.y)
        Lat_str, Lon_str     = str(Lat), str(lon)
        kochi                = ephem.Observer()
        kochi.lat, kochi.lon = Lat_str, Lon_str
        kochi.date           = Date
        kochi.date          -= ephem.hour * 9
        lt                   = ephem.localtime(kochi.date)
        sun                  = ephem.Sun(kochi)
        sun.compute(kochi)
        rhl                  = sun.hlon - ephem.degrees('180:00:00')
        dhl                  = ephem.degrees(rhl).norm

        # 太陽方位・高度を10進数に変換
        sun_alt     = float(str(sun.alt).split(':')[0]) + float(str(sun.alt).split(':')[1]) / 60+float(str(sun.alt).split(':')[2]) / 3600 #太陽高度
        sun_az      = float(str(sun.az).split(':')[0])  + float(str(sun.az).split(':')[1])  / 60+float(str(sun.az).split(':')[2])  / 3600 #太陽方位角
        #rad_sun_alt = math.radians(sun_alt)
        #rad_sun_azi = math.radians(sun_az)
        self.sunalt = sun_alt
        self.sunazl = sun_az

        # 光源両方まとめて計算させる      
        # 光源：入射光方向の計算（ここ重要）※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
        #rad_Incident_azPlus = math.radians(sun_az)  # 入射光方位
        # 光源：入射光方向の単位ベクトル
        #self.sun_xI         = math.sin(rad_Incident_azPlus) * math.cos(rad_sun_alt)
        #self.sun_yI         = math.cos(rad_Incident_azPlus) * math.cos(rad_sun_alt)
        #self.sun_zI         = math.sin(rad_sun_alt)
        #self.sun_vectorI    = math.sqrt((self.sun_xI**2)+(self.sun_yI**2)+(self.sun_zI**2)) #単位ベクトルの長さ
        # ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※

        # 光源両方まとめて計算させる      
        # 光源：入射光方向の計算（ここ重要）※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
        rad_sun_alt = math.radians(sun_alt)
        rad_sun_azi = math.radians(sun_az)

        sx             = math.sin(rad_sun_azi) * math.cos(rad_sun_alt)
        sy             = math.cos(rad_sun_azi) * math.cos(rad_sun_alt)
        sz             = math.sin(rad_sun_alt)
        self.vectorSun = np.array([sx, sy, sz])
        # ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※

    # 撮影時間を画像データから取得
    def PhotoTakenTime(self, PhotoFile):  
        exif_dict  = piexif.load(PhotoFile)
        for i,d in exif_dict["Exif"].items():
            if i == 36867:   # 日付が記載されているインデックス
                D = str(d)
        data       = (re.split('[: ]',D))
        Date  = data[0][2:6] +"/"+ data[1] +"/"+ data[2] +" "+ data[3] +":"+ data[4] +":"+ data[5][0:2]
        return Date

    # 関数１：画像情報をプリント
    def printInfo(self):
        print("\n","[ PHOTO INFO","]")
        print("PhotoID:",self.photoID, "  What line is written",self.num)
        print("x:",'{:.3f}'.format(self.x),"  y:",'{:.3f}'.format(self.y),"  z:",'{:.3f}'.format(self.z))
        print("Grand Resolution(mm)",'{:.3f}'.format(self.grand_reso), "  Width(m):",'{:.3f}'.format(self.areaW), "  Hight(m)",'{:.3f}'.format(self.areaH))
        print("::Image Range::"," Xmin(m)",'{:.3f}'.format(self.iXmin),"  Xmax(m)",'{:.3f}'.format(self.iXmax),"  Ymin(m)",'{:.3f}'.format(self.iYmin),"  Ymax(m)",'{:.3f}'.format(self.iYmax))
        print(self.sunalt ,self.sunazl)

### 関数セット:座標変換/オブリーク/位相角/観測角＋GeoTiff

In [25]:
###########################################################################
#############  XYZ座標からUV座標・ColRaw座標・画像座標の順で計算　  ##############
###########################################################################
def convertXYZtoCR(workload): # 関数使用条件：mapを使用する, arg1: workload
    
    PX,PY,PZ =     workload[0] ,     workload[1] ,     workload[2]
    PI,PJ,PK = int(workload[3]), int(workload[4]), int(workload[5])
    R, G, B  = 0,0,0 # RGBの0set

    arr1  = np.array([[PX-cx], [PY-cy], [PZ-cz]])
    arr7  = np.dot(carr6, arr1)

    # 共線条件式よりUVを算出
    U   = -sFocalLength * (arr7[0,0] / arr7[2,0])
    V   = -sFocalLength * (arr7[1,0] / arr7[2,0])
    
    # Col Row を算出
    col = round((U/sPixelSize)+iW/2+sCx,0).astype(int)
    row = round(iH/2-(V/sPixelSize)+sCy,0).astype(int)

    # RGBの取得
    if 0<=col<iW and 0<=row<iH and 0<=PI<=xsize and 0<=PJ<=ysize : # add shadow detection "and 1 == np_shadow[PJ,PI]"
            
            Rmin, Rmax = row-a, row+a+1
            Cmin, Cmax = col-a, col+a+1
            
            Rmin = 0      if Rmin < 0      else Rmin
            Rmax = iH     if Rmax > iH     else Rmax
            Cmin = 0      if Cmin < 0      else Cmin
            Cmax = iW     if Cmax > iW     else Cmax
            
            # NOTE 影の除去と標準偏差を加える場所はここ
            R, G, B = np.average(iR[Rmin:Rmax,Cmin:Cmax]), np.average(iG[Rmin:Rmax,Cmin:Cmax]), np.average(iB[Rmin:Rmax,Cmin:Cmax])
            #StdR, StdG, StdB = np.std(self.img_R[Rmin:Rmax,Cmin:Cmax]), np.std(self.img_G[Rmin:Rmax,Cmin:Cmax]), np.std(self.img_B[Rmin:Rmax,Cmin:Cmax])

    return [PI, PJ, PK, R, G, B]

###########################################################################
#########################        Occlusion       ##########################
###########################################################################
# 関数：Occlusion計算前の付属情報の整形処理  -------------------------------------
def DataFormatting1(OPa):
    OPb        = OPa[~np.all(OPa[:, 3:6] == 0, axis=1)]
    c_names   = ["PI", "PJ", "PK", "R", "G", "B"]
    DF        = pd.DataFrame(OPb,columns=c_names)
    DF["vi"]  = round(ci-DF["PI"],0).astype(int) #点とカメラの距離（X成分）
    DF["vj"]  = round(cj-DF["PJ"],0).astype(int) #点とカメラの距離（Y成分）
    DF["vk"]  = round(ck-DF["PK"],0).astype(int) #点とカメラの距離（Z成分）
    DF["d"]   = round(np.sqrt((DF["vi"])**2 + (DF["vj"])**2 + (DF["vk"])**2),0).astype(int)
    DF["T"]   = round(1 / DF["d"],8).astype(float)
    OPc       = DF.values
    return OPc
    
# 関数：XYZ座標からUV座標・ColRaw座標・画像座標の順で計算 -----------------------
def Occlusion8PhaseAngle(workload): # 関数使用条件：mapを使用する, arg1: workload
    
    PI,PJ,PK = int(workload[0]) , int(workload[1]) , int(workload[2])
    R,G,B    =     workload[3]  ,     workload[4]  ,     workload[5]
    vi,vj,vk =     workload[6]  ,     workload[7] ,      workload[8]
    d, T     = int(workload[9]),     workload[10]

    # 2点間周辺で最大標高を検索
    ista, iend = PI if PI <= ci else ci, PI if PI >= ci else ci
    jsta, jend = PJ if PJ <= cj else cj, PJ if PJ >= cj else cj
    Tmax = np.max(VoxelK[jsta:jend+1, ista:iend+1]) # +1は「start・end」が同値対応
    
    for n in range(0,d): # スタートを２個進んだところからにした
        
        # 地上から計算
        Tk, Tj, Ti = int(round(PK+vk*T*n,0)), int(round(PJ+vj*T*n,0)), int(round(PI+vi*T*n,0))
        
        # DEMより高くなると終わり
        if Tmax < Tk:
            break
        
        # 投影面の範囲から出ると終わり 
        elif xstart>Ti or Ti>xstop or ystart>Tj or Tj>ystop :
            R,G,B = 0,0,0
            break

        # もしTkよりDEMが高い場合は不可視
        moreHigher = VoxelK[Tj,Ti] > Tk
        if moreHigher==True:
            R,G,B = 0,0,0
            break
        #else:
        #    continue

    return [PI, PJ, PK, R, G, B]

###########################################################################
#################         Phase Angle（位相角度）        ####################
###########################################################################
# 関数：Phase Angle計算前の付属情報の整形処理  -------------------------------------
def DataFormatting2(OPa):
    OPb        = OPa[~np.all(OPa[:, 3:6] == 0, axis=1)]
    c_names   = ["PI", "PJ", "PK", "R", "G", "B"]
    DF        = pd.DataFrame(OPb,columns=c_names)
    DF["px"]  = round(DF["PI"]*Interval+xmin,5).astype(float)
    DF["py"]  = round(ymax-DF["PJ"]*Interval,5).astype(float)
    DF["pz"]  = round(DF["PK"]*Interval+zmin,5).astype(float)
    DF        = DF.drop(["PK","R", "G", "B"],axis=1)
    OPc       = DF.values
    return OPc

def calculate_azimuth_elevation(x, y, z):
    # elevation
    r = math.sqrt(x**2 + y**2 + z**2)          # 原点からの距離を計算
    elevation = math.degrees(math.asin(z / r)) # 仰角の計算
    
    # azimuth
    # 特殊な処理: x, y の符号によって角度を調整
    if x == 0:
        if y > 0:
            return 0, elevation
        elif y < 0:
            return 180, elevation
    elif y == 0:
        if x > 0:
            return 90, elevation
        elif x < 0:
            return 270, elevation
    
    # y 軸からの角度を計算
    azimuth = math.atan2(x, y)
    azimuth = math.degrees(azimuth)
    
    # 計算された角度を0から360度の範囲に変換
    azimuth = (azimuth + 360) % 360
    
    return azimuth,elevation

def PhaseAngle(workload):

    PI,PJ     = int(workload[0]) , int(workload[1])
    PX,PY,PZ =     workload[2] ,     workload[3] ,     workload[4]

    origin            = np.array([PX,PY,PZ])
    camera            = np.array([cx,cy,cz])

    vectorCam         = camera - origin

    dot_product       = np.dot(vectorCam,vectorSun)
    normSun           = np.linalg.norm(vectorSun)
    normCam           = np.linalg.norm(vectorCam)
    cosine_similarity = dot_product / (normCam*normSun)
    angle_in_radians  = np.arccos(cosine_similarity)
    angle_in_degrees  = np.degrees(angle_in_radians)

    #Observation Angle
    ox, oy, oz         = vectorCam
    azimuth, elevation = calculate_azimuth_elevation(ox, oy, oz)

    #Observation Angle To Normal
    Nxyz     =  np.linalg.norm(vectorCam)
    Normal   =  vectorCam / Nxyz
    Nx,Ny,Nz =  Normal

    return [PI, PJ, angle_in_degrees, azimuth, elevation, Nx, Ny, Nz]

###########################################################################
#########################        Geo Tiff　      ##########################
###########################################################################
# 関数：Geo Tiff Save（input for matrix） ---------------------------------
def GeoTiffSave3(ResultList1,ResultList2,OFile): 

    #位相角画像の２次元配列
    imageP_np = np.zeros((ysize, xsize)).astype(float)

    #仮想画像RGBそれぞれの２次元配列
    imageR_np = np.zeros((ysize, xsize)).astype(int)
    imageG_np = np.zeros((ysize, xsize)).astype(int)
    imageB_np = np.zeros((ysize, xsize)).astype(int)

    for p in ResultList2:
        i, j    = int(p[0]),  int(p[1])
        PA      = int(p[2])
        imageP_np[j,i] = PA

    for p in ResultList1:
        i, j    = int(p[0]),  int(p[1])
        r, g, b = int(p[3]), int(p[4]), int(p[5])

        if imageP_np[j,i] == 0:
            imageR_np[j,i], imageG_np[j,i], imageB_np[j,i] = 0,0,0
        else:
            imageR_np[j,i], imageG_np[j,i], imageB_np[j,i] = r, g, b

    # RGB の保存
    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile,xsize,ysize,3,gdal.GDT_UInt16)  # 画素数と型の設定
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))   # 位置情報の設定
    srs = osr.SpatialReference()                                  # 空間参照情報
    srs.ImportFromEPSG(2446)                                      # 日本測地第４系に座標系を指定
    dataset.SetProjection(srs.ExportToWkt())                      # 空間情報を結合
    dataset.GetRasterBand(1).WriteArray(imageR_np)
    dataset.GetRasterBand(2).WriteArray(imageG_np)
    dataset.GetRasterBand(3).WriteArray(imageB_np)
    dataset.FlushCache()
    dataset = None

# 関数：Geo Tiff Save（input for matrix） ---------------------------------
def GeoTiffSave1(ResultList,OFile): 

    #仮想画像RGBそれぞれの２次元配列
    imageP_np = np.zeros((ysize, xsize)).astype(float)

    for p in ResultList:
        i, j             = int(p[0]),  int(p[1])
        p                = p[2]
        imageP_np[j,i]   = p

     # Phase Angle の保存
    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile,xsize,ysize,1,gdal.GDT_Float32) # 画素数と型の設定
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))   # 位置情報の設定
    srs = osr.SpatialReference()                                  # 空間参照情報
    srs.ImportFromEPSG(2446)                                      # 日本測地第４系に座標系を指定
    dataset.SetProjection(srs.ExportToWkt())                      # 空間情報を結合
    dataset.GetRasterBand(1).WriteArray(imageP_np)
    dataset.FlushCache()
    dataset = None

# 関数：Geo Tiff Save to HSV and RGB Image（input for matrix） ---------------------------------
def GeoTiffSave3A(ResultList1,ResultList2,OFile1,OFile2): 

    #仮想画像HSVそれぞれの２次元配列
    hue_np        = np.zeros((ysize, xsize)).astype(float)
    saturation_np = np.zeros((ysize, xsize)).astype(float)
    value_np      = np.zeros((ysize, xsize)).astype(float)

    for p in ResultList1:
        i, j                            = int(p[0]), int(p[1])
        phase_angle, azimuth, elevation = p[2], p[3], p[4]

        if elevation >= 70:
            hue                             = azimuth   / 360
            saturation                      = elevation / 90
            hue_np[j,i], saturation_np[j,i] = hue , saturation


    for p in ResultList2:
        i, j    = int(p[0]),  int(p[1])
        r, g, b = int(p[3]), int(p[4]), int(p[5])
        
        if hue_np[j,i] == 0:
            value_np[j,i] = 0
        else:
            value_np[j,i] = max(r, g, b) / 65535 # Divide wit Maximum number in 16 bit

    # HSV の保存
    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile1,xsize,ysize,3,gdal.GDT_Float32)  # 画素数と型の設定
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))   # 位置情報の設定
    srs = osr.SpatialReference()                                  # 空間参照情報
    srs.ImportFromEPSG(2446)                                      # 日本測地第４系に座標系を指定
    dataset.SetProjection(srs.ExportToWkt())                      # 空間情報を結合
    dataset.GetRasterBand(1).WriteArray(hue_np)
    dataset.GetRasterBand(2).WriteArray(saturation_np)
    dataset.GetRasterBand(3).WriteArray(value_np)
    dataset.FlushCache()
    dataset = None

    # Convert HSV to RGB
    # 仮想画像RGBそれぞれの２次元配列
    R_np  = np.zeros((ysize, xsize)).astype(float)
    G_np  = np.zeros((ysize, xsize)).astype(float)
    B_np  = np.zeros((ysize, xsize)).astype(float)

    for j in range(0,ysize):
        for i in range(0,xsize):
            if value_np[j,i] > 0:
                R_np[j,i], G_np[j,i], B_np[j,i] = colorsys.hsv_to_rgb(hue_np[j,i],saturation_np[j,i],value_np[j,i])

    # RGB の保存
    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile2,xsize,ysize,3,gdal.GDT_Float32)  # 画素数と型の設定
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))     # 位置情報の設定
    srs = osr.SpatialReference()                                    # 空間参照情報
    srs.ImportFromEPSG(2446)                                        # 日本測地第４系に座標系を指定
    dataset.SetProjection(srs.ExportToWkt())                        # 空間情報を結合
    dataset.GetRasterBand(1).WriteArray(R_np)
    dataset.GetRasterBand(2).WriteArray(G_np)
    dataset.GetRasterBand(3).WriteArray(B_np)
    dataset.FlushCache()
    dataset = None

# 関数：Geo Tiff Save（input for matrix） ---------------------------------
def GeoTiffSave3N(ResultList1,OFile): 

    #単位ベクトルそれぞれの２次元配列
    imageNx_np   = np.zeros((ysize, xsize)).astype(float)
    imageNy_np   = np.zeros((ysize, xsize)).astype(float)
    imageNz_np   = np.zeros((ysize, xsize)).astype(float)

    for p in ResultList1:
        i, j      = int(p[0]),  int(p[1])
        elevation = p[4]
        Nx,Ny,Nz  = p[5],p[6],p[7]

        if elevation >= 70:
            imageNx_np[j,i] = np.round(((Nx + 1)/2) * 255,0)
            imageNy_np[j,i] = np.round(((Ny + 1)/2) * 255,0)
            imageNz_np[j,i] = np.round(((Nz + 1)/2) * 255,0)

    # Convert To Blue Map
    #imageNx_np = np.round(((imageNx_np + 1)/2) * 255,0)
    #imageNy_np = np.round(((imageNy_np + 1)/2) * 255,0)
    #imageNz_np = np.round(((imageNz_np + 1)/2) * 255,0)

    # Save to Tiff file
    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile,xsize,ysize,3,gdal.GDT_Byte)  # 画素数と型の設定 #gdal.GDT_Byte , gdal.GDT_Float32
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))   # 位置情報の設定
    srs = osr.SpatialReference()                                  # 空間参照情報
    srs.ImportFromEPSG(2446)                                      # 日本測地第４系に座標系を指定
    dataset.SetProjection(srs.ExportToWkt())                      # 空間情報を結合
    dataset.GetRasterBand(1).WriteArray(imageNx_np)
    dataset.GetRasterBand(2).WriteArray(imageNy_np)
    dataset.GetRasterBand(3).WriteArray(imageNz_np)
    dataset.FlushCache()
    dataset = None
    


###########################################################################
#######################        Shadow Tiff　      #########################
###########################################################################
# 関数：日向日陰データの読み込み -----------------------------------------------
def ShadowTiff(IFile):
    ti_shadow = tifffile.imread(IFile)
    np_shadow = np.array(ti_shadow[:,:]).astype(int)
    return np_shadow

## 並列処理セット

In [26]:
###########################################################################
##################  　　　　　　　共用できる関数　　　　　　　   #################
###########################################################################
# 関数1：並列処理のための仕事量分割量の計算 --------------------------------------
def Workers(P_np):
    number , column = P_np.shape
    workers      = int(os.cpu＿count())
    workloadRow  = number//workers  # 切り下げ
    return number, workers, workloadRow


###########################################################################
##################　投影面からUV-ColRawを計算する座標計算に利用 #################
###########################################################################
# 関数1：スレッドに仕事を均等に渡す処理 ------------------------------------------
def parallelprocessing(P_np, number, workers, workloadRow, wnumber): # forで仕事をworkerに渡す
    start      = workloadRow * wnumber
    end        = number + 1 if wnumber == workers else workloadRow * (wnumber + 1)
    workload   = P_np[start:end]
    resultlist = list(map(convertXYZtoCR,workload))
    return resultlist

# 関数2：投影面DEMから並列処理により原画像のUVを求める -----------------------------
def Confu(self):
    # 並列処理によって「投影面DEMのxyzから現画像uvを計算」そして保存
    with confu.ProcessPoolExecutor(max_workers=os.cpu＿count()) as executor: 
        number, workers, workloadRow = Workers(P_np)
        futures = [executor.submit(parallelprocessing, P_np, number, workers, workloadRow, wnumber) for wnumber in range(0,workers)]

        # 結果をまとめる
        marge_PointList = np.array([[0,1,2,3,4,5]])
        for f in tqdm(futures):
            result_XYZUVIJK = f.result()
            marge_PointList  = np.concatenate([marge_PointList, result_XYZUVIJK],0)

    #結果を整形してから保存する
    PointsList2Bsaved = np.delete(marge_PointList, 0, axis=0)
    return PointsList2Bsaved


###########################################################################
#####################   　オクルージョンの算出に利用     #######################
###########################################################################
# 関数1：スレッドに仕事を均等に渡す処理 ------------------------------------------
def parallelprocessing_2(OP2, number, workers, workloadRow, wnumber): # forで仕事をworkerに渡す
    start      = workloadRow * wnumber
    end        = number + 1 if wnumber == workers else workloadRow * (wnumber + 1)
    workload   = OP2[start:end]
    resultlist = list(map(Occlusion8PhaseAngle,workload))
    return resultlist

# 関数2：投影面DEMから並列処理により原画像のUVを求める -----------------------------
def Confu_2(OP2):
    # 並列処理によって「投影面DEMのxyzから現画像uvを計算」そして保存
    with confu.ProcessPoolExecutor(max_workers=os.cpu＿count()) as executor: 
        number, workers, workloadRow = Workers(OP2)
        futures = [executor.submit(parallelprocessing_2, OP2, number, workers, workloadRow, wnumber) for wnumber in range(0,workers)]

        # 結果をまとめる
        marge_PointList = np.array([[0,1,2,3,4,5]])
        for f in tqdm(futures):
            result_IJKRGB = f.result()
            marge_PointList  = np.concatenate([marge_PointList, result_IJKRGB],0)

    #結果を整形してから保存する
    PointsList2Bsaved = np.delete(marge_PointList, 0, axis=0)
    return PointsList2Bsaved


###########################################################################
########################       位相角の算出に利用      #######################
###########################################################################
# 関数1：スレッドに仕事を均等に渡す処理 ------------------------------------------
def parallelprocessing_3(OP4, number, workers, workloadRow, wnumber): # forで仕事をworkerに渡す
    start      = workloadRow * wnumber
    end        = number + 1 if wnumber == workers else workloadRow * (wnumber + 1)
    workload2  = OP4[start:end]
    resultlist = list(map(PhaseAngle,workload2))
    return resultlist

# 関数2：投影面DEMから並列処理により原画像のUVを求める -----------------------------
def Confu_3(OP4):
    # 並列処理によって「投影面DEMのxyzから現画像uvを計算」そして保存
    with confu.ProcessPoolExecutor(max_workers=os.cpu＿count()) as executor: 
        number, workers, workloadRow = Workers(OP4)
        futures = [executor.submit(parallelprocessing_3, OP4, number, workers, workloadRow, wnumber) for wnumber in range(0,workers)]

        # 結果をまとめる
        marge_PointList = np.array([[0,1,2,3,4,5,6,7]])
        for f in tqdm(futures):
            result_IJPa = f.result()
            marge_PointList  = np.concatenate([marge_PointList, result_IJPa],0)

    #結果を整形してから保存する
    PointsList2Bsaved = np.delete(marge_PointList, 0, axis=0)
    return PointsList2Bsaved


### 画像の隅っこがオクルージョンでケラられて、RGB値しか無い場合の対応

In [27]:
def DeleteRGB(OP3,OP5):
    R_np2  = np.zeros((ysize, xsize)).astype(float)
    G_np2  = np.zeros((ysize, xsize)).astype(float)
    B_np2  = np.zeros((ysize, xsize)).astype(float)

    for j in range(0,ysize):
        for i in range(0,xsize):
            if value_np[j,i] > 0:
                R_np2[j,i], G_np2[j,i], B_np2[j,i] = colorsys.hsv_to_rgb(hue_np[j,i],saturation_np[j,i],value_np[j,i])
    return

# ||||プログラム実行

In [28]:
###############  条件  ###############
# 観測仰角が90から70度までを計算する #
######################################

# SensorParameter
Sensor = SensorParameter(CameraName)

sPixelSize   = Sensor.pixel_size
sFocalLength = Sensor.focal_length
sCx , sCy    = Sensor.cX, Sensor.cY
siFov        = Sensor.iFov
si_rX, si_rY = Sensor.i_rX,Sensor.i_rY
VoxelK       = Sensor.VoxelK
xstart, ystart, xstop, ystop = Sensor.xstart, Sensor.ystart, Sensor.xstop, Sensor.ystop

Sensor.printInfo()
print(CameraName)
#np_shadow = ShadowTiff(IFile6)

for i in PhotoIDs:
    start  = time.time() # 計算時間を計る
    print(i,end=" ")
    # ImageParameter
    Image                = CameraParameterEach(i)
    iH, iW               = int(Image.h), int(Image.w)
    iR, iG, iB           = Image.img_R,Image.img_G,Image.img_B
    cx, cy, cz           = Image.x, Image.y, Image.z
    ci, cj, ck           = Image.Ci, Image.Cj, Image.Ck
    carr6                = Image.arr6
    vectorSun            = Image.vectorSun
    OFile1,OFile2,OFile3 = Image.OFile1, Image.OFile2, Image.OFile3
    OFile4,OFile5        = Image.OFile4, Image.OFile5

    print(Image.sunalt,Image.sunazl,Image.vectorSun)
    #Image.printInfo()

    if xstart > ci or ci > xstop or ystart > cj or cj > ystop: # 範囲外で飛行した場合は対象外
        df = pd.read_csv(IFile5, header=None)                  # CSVファイルを読み込み
        df = df[df[0] != i]                                    # i を含む行を削除
        df.to_csv(IFile5, index=False, header=False)           # CSVファイルに保存
        print(": outside the area , remove from list")
        continue

    # オルソフォト作成
    OP1 = Confu(P_np) # P_np（投影面）

    # オクルージョン処理
    OP2 = DataFormatting1(OP1)
    OP3 = Confu_2(OP2)
    
    # 位相角
    OP4 = DataFormatting2(OP3)
    OP5 = Confu_3(OP4)

    # SaveImage # Output only the parts where all values are correct
    GeoTiffSave3(OP3,OP5,OFile1)             # Oltho Photo
    GeoTiffSave1(OP5,OFile2)                 # Phase Angle
    GeoTiffSave3A(OP5,OP3,OFile3,OFile4)     # Observation Angle HSV
    GeoTiffSave3N(OP5,OFile5)                # Observation Angle Blue Map

    del OP1
    del OP2
    del OP3
    del OP4
    del OP5

    print ("elapsed_time:{0}".format(time.time()-start) + "[sec]") # 計算時間を表示


 [ Sensor Info ]
Sensor Nmae: P1   Pixel Width: 8192   Pixel Hight: 5460
Pixel Size: 0.0000043948614 Cx: -34.02808  Cy: 21.09109
Sensor SizeX(m) 0.03600   Sensor SizeY(m) 0.02400
Focal Length: 7973.36592  Camera iFOV: 0.125
P1
DJI_20231228115022_0001 33.38458333333333 176.04663888888888 [ 0.05756832 -0.83300908  0.55025609]


100%|██████████| 64/64 [00:02<00:00, 30.16it/s]


elapsed_time:33.16461181640625[sec]
DJI_20231228115025_0002 33.385305555555554 176.06033333333335 [ 0.05736874 -0.8330159   0.55026661]


100%|██████████| 64/64 [00:02<00:00, 31.46it/s]


elapsed_time:33.66990566253662[sec]
DJI_20231228115028_0003 33.38602777777778 176.07399999999998 [ 0.05716957 -0.83302264  0.55027714]


...

elapsed_time:33.37090086936951[sec]
DJI_20231228115633_0147 33.45447222222222 177.7446388888889 [ 0.03283342 -0.83367783  0.5512742 ]


100%|██████████| 64/64 [00:02<00:00, 26.30it/s]


elapsed_time:34.77412486076355[sec]
DJI_20231228115636_0148 33.45488888888889 177.75841666666668 [ 0.03263279 -0.8336817   0.55128026]


100%|██████████| 64/64 [00:02<00:00, 30.71it/s]


elapsed_time:33.42312669754028[sec]


In [29]:
# 単位ベクトルの計算
x,y,z    = 4,5,9
xyz      =  np.array([x,y,z])
xyzN     =  np.linalg.norm(xyz)
u        =  xyz / xyzN
Nx,Ny,Nz = u

print(Nx,Ny,Nz)

0.3621429841700741 0.45267873021259264 0.8148217143826667


In [30]:
x,y,z    = 4,5,9
vectorCam = np.array([x,y,z])
Nxyz     =  np.linalg.norm(vectorCam)
Normal   =  vectorCam / Nxyz
Nx,Ny,Nz = Normal

print(Nx,Ny,Nz)

0.3621429841700741 0.45267873021259264 0.8148217143826667
